<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/Building_a_Chatbot.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Building a Chatbot

In this notebook, you’ll build a chatbot using Cohere’s Chat endpoint. By the end of this notebook, you’ll be able to build a simple chatbot that can respond to user messages and maintain the context of the conversation.

*Read the accompanying [blog post here](https://docs.cohere.com/docs/building-a-chatbot).*

## Overview

We'll do the following steps:
- **Step 1: Quickstart** - Learn the quickest and easiest way to call the Chat endpoint.
- **Step 2: Adding a preamble** - Steer a chatbot's response toward certain styles, personas, or other characteristics.
- **Step 3: Streaming the response** - Display a chatbot's response incrementally as it is generated, as opposed to waiting for the entire response to be completed.
- **Step 4: Building the Chat History** - Explore two different options for getting the chatbot to maintain the context of the conversation.

## Setup

We'll start by installing the tools we'll need and then importing them.

In [1]:
! pip install cohere -q

In [2]:
#@title Enable text wrapping in Google Colab

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import uuid
import cohere
import json
from cohere import ChatMessage

Fill in your Cohere API key in the next cell. To do this, begin by [signing up to Cohere](https://os.cohere.ai/) (for free!) if you haven't yet. Then get your API key [here](https://dashboard.cohere.com/api-keys).

In [4]:
# Paste your API key here. Remember to not share publicly
co = cohere.Client("COHERE_API_KEY") 

## Step 1: Quickstart

The `message` parameter is the only required parameter for the Chat endpoint. 

In our case, we also use the `model` parameter to specify the underlying chat model we want to use. For this, we pick Command R+, Cohere's newest large language model.

We call the endpoint with `co.chat()`, and get the main content of the response from the `text` value.

In [5]:
response = co.chat(message="Hello", model="command-r-plus")
print(response.text)

Hello! How can I help you today?


## Step 2: Adding a preamble

A conversation starts with a system message, or a preamble, to help steer a chatbot’s response toward certain characteristics. In the quickstart example, we didn’t have to define a preamble because a default one was used. 

We can define our own preamble with the `preamble` parameter.

In [12]:
response = co.chat(message="Hello",
                   model="command-r-plus",
                   preamble="You are an expert public speaking coach. Don't use any greetings.")
print(response.text)

How can I help you with your public speaking today?


## Step 3: Streaming the response

Our examples so far generate responses in a non-streamed manner. This means that the endpoint would return a response object only after the model has generated the text in full. 

The Chat endpoint also supports streamed responses, where you can display the text incrementally without having to wait for the full completion. To activate it, use `co.chat_stream()` instead of `co.chat()`.

In streaming mode, the endpoint will generate a series of objects. To get the text content, we take objects whose `event_type` is `text-generation`.

In [8]:
stream = co.chat_stream(message="Hello. I'd like to learn about techniques for effective audience engagement",
                        model="command-r-plus",
                        preamble="You are an expert public speaking coach")

for event in stream:
    if event.event_type == "text-generation":
        print(event.text, end='')

Sure! Here are some techniques for effective audience engagement:

- **Make eye contact:** Look at different individuals in the audience and try to maintain eye contact for a few seconds at a time. This helps create a sense of connection and makes your speech more intimate.

- **Use gestures:** Incorporate hand gestures and body language to emphasize your points and add variety to your delivery. This helps keep your audience engaged and can also help you appear more confident and passionate about your topic.

- **Vary your tone and volume:** Avoid monotone delivery by varying your pitch, pace, and volume. Emphasize important points by raising your volume or slowing down your pace. This helps to create emphasis and keeps your audience interested.

- **Ask questions:** Pose rhetorical questions to your audience to get them thinking about your topic. You can also directly ask for their opinions or input, either by a show of hands or by inviting them to share their thoughts. This creates a

## Step 4: Building the Chat history

At the core of a conversation is a multi-turn dialog between the user and the chatbot. This requires the chatbot to have a “memory” of all the previous turns to maintain the state of the conversation.

### Option 1: Using the conversation history persistence feature

The Chat endpoint supports state management by persisting the conversation history. As a conversation progresses, the endpoint continuously updates the conversation history. This means developers don’t have to deal with the complexity and inconvenience of managing conversation history in their application.

To use this feature, use the `conversation_id` parameter, which is a unique string you assign to a conversation. We'll use the `uuid` library to do this.

Putting everything together, let’s now build a simple chat interface that takes in a user message, generates the chatbot response, automatically updates the conversation history, and repeats these steps until the user quits the conversation. 

As described before, in streaming mode, the Chat endpoint generates a series of objects. To get the conversation history, we take the object with `event_type` of `"stream-end"` and save it as a new variable `chat_history`.

In [13]:
# Create a conversation ID
conversation_id = str(uuid.uuid4())

# Define the preamble
preamble = "You are an expert public speaking coach"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                            model="command-r-plus",
                            preamble=preamble,
                            conversation_id=conversation_id)

    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')
        if event.event_type == "stream-end":
            chat_history = event.response.chat_history

    print(f"\n{'-'*100}\n")

Starting the chat. Type "quit" to end.

User: Hello
Chatbot: Hello! How can I help you today?
----------------------------------------------------------------------------------------------------

User: I'd like to learn about techniques for effective audience engagement
Chatbot: Sure! Here are some techniques for effective audience engagement:

- Make eye contact with your audience and use hand gestures to emphasize your points.
- Use a clear and concise language that your audience can easily understand.
- Vary your tone and volume to keep your audience interested.
- Use visual aids such as slides, props, or demonstrations to help illustrate your points.
- Tell stories or anecdotes to help your audience connect with your message on a personal level.
- Ask questions to involve your audience and encourage them to think about your topic.
- Use humor appropriately to keep your audience entertained and engaged.
- Be enthusiastic and passionate about your topic to show your audience that you

Next, we print the full conversation history.

In [14]:
for chat in chat_history:
    print(chat)

role='USER' message='Hello' generation_id='b4146b59-0884-4aa3-a573-bb6bd4948511' response_id='b72e9f29-ac13-41e2-8892-b6529f9547f2'
role='CHATBOT' message='Hello! How can I help you today?' generation_id='3a002415-0213-49b9-8d4b-b7c24eba95f8' response_id='b72e9f29-ac13-41e2-8892-b6529f9547f2'
role='USER' message="I'd like to learn about techniques for effective audience engagement" generation_id='fee7fbf4-fd93-4477-b987-bd2765694962' response_id='520e2329-979b-4e4b-9cdc-b2e4b49c3612'
role='CHATBOT' message="Sure! Here are some techniques for effective audience engagement:\n\n- Make eye contact with your audience and use hand gestures to emphasize your points.\n- Use a clear and concise language that your audience can easily understand.\n- Vary your tone and volume to keep your audience interested.\n- Use visual aids such as slides, props, or demonstrations to help illustrate your points.\n- Tell stories or anecdotes to help your audience connect with your message on a personal level.\n

### Option 2: Managing the conversation history yourself

If you opt not to use the endpoint’s conversation history persistence feature, you can use the `chat_history` parameter to manage the conversation history yourself. 

In [15]:
# Initialize the chat history
chat_history = []

# Define the preamble
preamble = "You are an expert public speaking coach"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                            model="command-r-plus",
                            preamble=preamble,
                            chat_history=chat_history)

    chatbot_response = ""
    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')
            chatbot_response += event.text
    print("\n")

    # Add to chat history
    chat_history.extend(
        [ChatMessage(role="USER", message=message),
         ChatMessage(role="CHATBOT", message=chatbot_response)]
    )

Starting the chat. Type "quit" to end.

User: Hello
Chatbot: Hello! How can I help you today?

User: I'd like to learn about techniques for effective audience engagement
Chatbot: Sure! Here are some techniques for effective audience engagement:

- Make eye contact with your audience and use hand gestures to emphasize your points.
- Use a clear and concise language that your audience can easily understand.
- Vary your tone and volume to keep your audience interested.
- Use stories, examples, and analogies to illustrate your points and make your message more memorable.
- Ask questions to involve your audience and encourage them to think about your topic.
- Use humor appropriately to lighten the mood and engage your audience.
- Use visuals, such as slides, photos, or props, to break up your speech and add interest.
- Move around the stage or room to change your position and keep your audience focused.
- Use pauses to emphasize important points and give your audience time to absorb your me

We confirm that `chat_history` is consistent with the conversation above.

In [16]:
for chat in chat_history:
    print(chat)

role='USER' message='Hello'
role='CHATBOT' message='Hello! How can I help you today?'
role='USER' message="I'd like to learn about techniques for effective audience engagement"
role='CHATBOT' message='Sure! Here are some techniques for effective audience engagement:\n\n- Make eye contact with your audience and use hand gestures to emphasize your points.\n- Use a clear and concise language that your audience can easily understand.\n- Vary your tone and volume to keep your audience interested.\n- Use stories, examples, and analogies to illustrate your points and make your message more memorable.\n- Ask questions to involve your audience and encourage them to think about your topic.\n- Use humor appropriately to lighten the mood and engage your audience.\n- Use visuals, such as slides, photos, or props, to break up your speech and add interest.\n- Move around the stage or room to change your position and keep your audience focused.\n- Use pauses to emphasize important points and give your

And with that, we have built a simple chatbot that can respond to user messages and maintain the context of the conversation.